In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 10000*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240320013141
0	Validation losses: 0.4311, 286.8460, 287.2771	Best Loss: 287.2771 (0)	Accuracy: 83.48%
1	Validation losses: 0.3566, 266.5747, 266.9313	Best Loss: 266.9313 (0)	Accuracy: 87.56%
2	Validation losses: 0.3591, 254.9233, 255.2824	Best Loss: 255.2824 (0)	Accuracy: 86.80%
3	Validation losses: 0.3512, 246.4709, 246.8220	Best Loss: 246.8220 (0)	Accuracy: 86.90%
4	Validation losses: 0.3331, 239.6263, 239.9594	Best Loss: 239.9594 (0)	Accuracy: 87.34%
5	Validation losses: 0.3769, 233.7901, 234.1670	Best Loss: 234.1670 (0)	Accuracy: 85.56%
6	Validation losses: 0.2966, 228.3501, 228.6467	Best Loss: 228.6467 (0)	Accuracy: 89.04%
7	Validation losses: 0.3428, 223.1162, 223.4590	Best Loss: 223.4590 (0)	Accuracy: 86.94%
8	Validation losses: 0.2969, 217.7669, 218.0639	Best Loss: 218.0639 (0)	Accuracy: 89.12%
9	Validation losses: 0.3861, 213.4808, 213.8669	Best Loss: 213.8669 (0)	Accuracy: 87.20%
10	Validation losses: 0.2866, 208.8178, 209.1044	Best

24	Validation losses: nan, nan, nan	Best Loss: 209.8262 (14)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 209.8262 (15)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 209.8262 (16)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 209.8262 (17)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 209.8262 (18)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 209.8262 (19)	Accuracy: 0.00%
30	Validation losses: nan, nan, nan	Best Loss: 209.8262 (20)	Accuracy: 0.00%
Early stopping!
Total running time:	 2546
INFO:tensorflow:Restoring parameters from ./models/model_20240320030511.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.1800005436%
RANDOM SEED:   1 	LAMBDA_EXP:   -4.2
Now: 20240320034739
0	Validation losses: 0.4230, 186.6452, 187.0682	Best Loss: 187.0682 (0)	Accuracy: 83.88%
1	Validation losses: 0.3613, 172.899

19	Validation losses: 0.4669, 112.8284, 113.2953	Best Loss: 113.2953 (0)	Accuracy: 87.54%
20	Validation losses: 0.3259, 110.3115, 110.6373	Best Loss: 110.6373 (0)	Accuracy: 89.42%
21	Validation losses: 0.3215, 108.5910, 108.9125	Best Loss: 108.9125 (0)	Accuracy: 89.36%
22	Validation losses: 0.3663, 107.0710, 107.4373	Best Loss: 107.4373 (0)	Accuracy: 89.92%
23	Validation losses: 0.3726, 106.2075, 106.5801	Best Loss: 106.5801 (0)	Accuracy: 89.46%
24	Validation losses: 0.3713, 105.0524, 105.4236	Best Loss: 105.4236 (0)	Accuracy: 90.16%
25	Validation losses: 0.3769, 104.3111, 104.6879	Best Loss: 104.6879 (0)	Accuracy: 89.40%
26	Validation losses: 0.3898, 103.2615, 103.6513	Best Loss: 103.6513 (0)	Accuracy: 90.16%
27	Validation losses: 0.3970, 102.7744, 103.1714	Best Loss: 103.1714 (0)	Accuracy: 90.20%
28	Validation losses: 0.4489, 102.1579, 102.6068	Best Loss: 102.6068 (0)	Accuracy: 90.58%
29	Validation losses: 0.4270, 101.6813, 102.1083	Best Loss: 102.1083 (0)	Accuracy: 90.50%
30	Validat

112	Validation losses: 0.8604, 95.7374, 96.5978	Best Loss: 96.5978 (0)	Accuracy: 90.96%
113	Validation losses: 0.8613, 95.7310, 96.5922	Best Loss: 96.5922 (0)	Accuracy: 90.94%
114	Validation losses: 0.8622, 95.7237, 96.5859	Best Loss: 96.5859 (0)	Accuracy: 90.94%
115	Validation losses: 0.8630, 95.7169, 96.5800	Best Loss: 96.5800 (0)	Accuracy: 90.96%
116	Validation losses: 0.8637, 95.7118, 96.5754	Best Loss: 96.5754 (0)	Accuracy: 90.94%
117	Validation losses: 0.8643, 95.7073, 96.5716	Best Loss: 96.5716 (0)	Accuracy: 90.96%
118	Validation losses: 0.8648, 95.7027, 96.5675	Best Loss: 96.5675 (0)	Accuracy: 90.96%
119	Validation losses: 0.8652, 95.6991, 96.5644	Best Loss: 96.5644 (0)	Accuracy: 90.96%
120	Validation losses: 0.8657, 95.6961, 96.5618	Best Loss: 96.5618 (0)	Accuracy: 90.94%
121	Validation losses: 0.8661, 95.6938, 96.5599	Best Loss: 96.5599 (0)	Accuracy: 90.94%
122	Validation losses: 0.8665, 95.6908, 96.5572	Best Loss: 96.5572 (0)	Accuracy: 90.94%
123	Validation losses: 0.8668, 9

206	Validation losses: 0.9101, 95.3550, 96.2651	Best Loss: 96.2651 (0)	Accuracy: 91.06%
207	Validation losses: 0.9103, 95.3525, 96.2628	Best Loss: 96.2628 (0)	Accuracy: 91.06%
208	Validation losses: 0.9105, 95.3497, 96.2602	Best Loss: 96.2602 (0)	Accuracy: 91.06%
209	Validation losses: 0.9108, 95.3477, 96.2585	Best Loss: 96.2585 (0)	Accuracy: 91.06%
210	Validation losses: 0.9111, 95.3448, 96.2559	Best Loss: 96.2559 (0)	Accuracy: 91.06%
211	Validation losses: 0.9114, 95.3431, 96.2545	Best Loss: 96.2545 (0)	Accuracy: 91.04%
212	Validation losses: 0.9118, 95.3413, 96.2530	Best Loss: 96.2530 (0)	Accuracy: 91.02%
213	Validation losses: 0.9121, 95.3395, 96.2516	Best Loss: 96.2516 (0)	Accuracy: 91.02%
214	Validation losses: 0.9123, 95.3378, 96.2501	Best Loss: 96.2501 (0)	Accuracy: 91.02%
215	Validation losses: 0.9127, 95.3360, 96.2488	Best Loss: 96.2488 (0)	Accuracy: 91.02%
216	Validation losses: 0.9131, 95.3337, 96.2468	Best Loss: 96.2468 (0)	Accuracy: 91.04%
217	Validation losses: 0.9134, 9

...

21	Validation losses: 0.3333, 1.0608, 1.3941	Best Loss: 1.3941 (0)	Accuracy: 88.54%
22	Validation losses: 0.3791, 1.0456, 1.4247	Best Loss: 1.3941 (1)	Accuracy: 88.12%
23	Validation losses: 0.3296, 1.0372, 1.3668	Best Loss: 1.3668 (0)	Accuracy: 89.52%
24	Validation losses: 0.3257, 1.0234, 1.3491	Best Loss: 1.3491 (0)	Accuracy: 89.56%
25	Validation losses: 0.3353, 1.0172, 1.3526	Best Loss: 1.3491 (1)	Accuracy: 89.68%
26	Validation losses: 0.3467, 1.0023, 1.3490	Best Loss: 1.3490 (0)	Accuracy: 89.60%
27	Validation losses: 0.3515, 0.9942, 1.3457	Best Loss: 1.3457 (0)	Accuracy: 89.46%
28	Validation losses: 0.3943, 0.9855, 1.3798	Best Loss: 1.3457 (1)	Accuracy: 90.08%
29	Validation losses: 0.3501, 0.9797, 1.3298	Best Loss: 1.3298 (0)	Accuracy: 90.20%
30	Validation losses: 0.3931, 0.9738, 1.3670	Best Loss: 1.3298 (1)	Accuracy: 90.04%
31	Validation losses: 0.3897, 0.9684, 1.3581	Best Loss: 1.3298 (2)	Accuracy: 90.34%
32	Validation losses: 0.4290, 0.9606, 1.3896	Best Loss: 1.3298 (3)	Accuracy:

17	Validation losses: 0.3165, 0.7506, 1.0671	Best Loss: 1.0671 (0)	Accuracy: 89.46%
18	Validation losses: 0.4335, 0.7442, 1.1777	Best Loss: 1.0671 (1)	Accuracy: 86.60%
19	Validation losses: 0.3439, 0.7366, 1.0805	Best Loss: 1.0671 (2)	Accuracy: 88.94%
20	Validation losses: 0.3412, 0.7414, 1.0826	Best Loss: 1.0671 (3)	Accuracy: 88.04%
21	Validation losses: 0.3511, 0.7228, 1.0739	Best Loss: 1.0671 (4)	Accuracy: 89.08%
22	Validation losses: 0.3381, 0.7140, 1.0521	Best Loss: 1.0521 (0)	Accuracy: 89.10%
23	Validation losses: 0.3533, 0.7027, 1.0561	Best Loss: 1.0521 (1)	Accuracy: 89.12%
24	Validation losses: 0.3220, 0.7002, 1.0222	Best Loss: 1.0222 (0)	Accuracy: 89.60%
25	Validation losses: 0.3328, 0.6874, 1.0202	Best Loss: 1.0202 (0)	Accuracy: 90.00%
26	Validation losses: 0.3634, 0.6796, 1.0430	Best Loss: 1.0202 (1)	Accuracy: 90.14%
27	Validation losses: 0.3790, 0.6749, 1.0539	Best Loss: 1.0202 (2)	Accuracy: 89.38%
28	Validation losses: 0.3736, 0.6704, 1.0440	Best Loss: 1.0202 (3)	Accuracy:

11	Validation losses: 0.3240, 0.5370, 0.8609	Best Loss: 0.8557 (1)	Accuracy: 88.88%
12	Validation losses: 0.3842, 0.5301, 0.9143	Best Loss: 0.8557 (2)	Accuracy: 86.74%
13	Validation losses: 0.3945, 0.5232, 0.9176	Best Loss: 0.8557 (3)	Accuracy: 86.44%
14	Validation losses: 0.3065, 0.5179, 0.8244	Best Loss: 0.8244 (0)	Accuracy: 88.92%
15	Validation losses: 0.3228, 0.5131, 0.8359	Best Loss: 0.8244 (1)	Accuracy: 88.68%
16	Validation losses: 0.3420, 0.5078, 0.8498	Best Loss: 0.8244 (2)	Accuracy: 88.62%
17	Validation losses: 0.3300, 0.5056, 0.8356	Best Loss: 0.8244 (3)	Accuracy: 89.08%
18	Validation losses: 0.3073, 0.5020, 0.8093	Best Loss: 0.8093 (0)	Accuracy: 89.70%
19	Validation losses: 0.3843, 0.4986, 0.8829	Best Loss: 0.8093 (1)	Accuracy: 88.50%
20	Validation losses: 0.3506, 0.4946, 0.8452	Best Loss: 0.8093 (2)	Accuracy: 88.36%
21	Validation losses: 0.3550, 0.4892, 0.8442	Best Loss: 0.8093 (3)	Accuracy: 87.66%
22	Validation losses: 0.3180, 0.4835, 0.8015	Best Loss: 0.8015 (0)	Accuracy:

12	Validation losses: 0.3658, 0.5298, 0.8956	Best Loss: 0.8611 (4)	Accuracy: 87.06%
13	Validation losses: 0.3334, 0.5232, 0.8566	Best Loss: 0.8566 (0)	Accuracy: 88.04%
14	Validation losses: 0.3163, 0.5170, 0.8333	Best Loss: 0.8333 (0)	Accuracy: 88.50%
15	Validation losses: 0.3176, 0.5126, 0.8302	Best Loss: 0.8302 (0)	Accuracy: 88.60%
16	Validation losses: 0.3160, 0.5099, 0.8259	Best Loss: 0.8259 (0)	Accuracy: 88.48%
17	Validation losses: 0.3243, 0.5037, 0.8280	Best Loss: 0.8259 (1)	Accuracy: 89.70%
18	Validation losses: 0.3511, 0.5007, 0.8518	Best Loss: 0.8259 (2)	Accuracy: 88.38%
19	Validation losses: 0.3677, 0.4972, 0.8649	Best Loss: 0.8259 (3)	Accuracy: 88.70%
20	Validation losses: 0.3657, 0.4963, 0.8620	Best Loss: 0.8259 (4)	Accuracy: 87.38%
21	Validation losses: 0.3334, 0.4883, 0.8217	Best Loss: 0.8217 (0)	Accuracy: 89.14%
22	Validation losses: 0.4048, 0.4828, 0.8876	Best Loss: 0.8217 (1)	Accuracy: 88.74%
23	Validation losses: 0.3586, 0.4759, 0.8346	Best Loss: 0.8217 (2)	Accuracy:

22	Validation losses: 0.3441, 0.3281, 0.6722	Best Loss: 0.6419 (7)	Accuracy: 89.64%
23	Validation losses: 0.3533, 0.3250, 0.6782	Best Loss: 0.6419 (8)	Accuracy: 90.06%
24	Validation losses: 0.3339, 0.3214, 0.6553	Best Loss: 0.6419 (9)	Accuracy: 89.62%
25	Validation losses: 0.3469, 0.3189, 0.6658	Best Loss: 0.6419 (10)	Accuracy: 89.74%
26	Validation losses: 0.3493, 0.3157, 0.6650	Best Loss: 0.6419 (11)	Accuracy: 90.24%
27	Validation losses: 0.3742, 0.3136, 0.6878	Best Loss: 0.6419 (12)	Accuracy: 90.08%
28	Validation losses: 0.3926, 0.3113, 0.7040	Best Loss: 0.6419 (13)	Accuracy: 90.28%
29	Validation losses: 0.4177, 0.3083, 0.7260	Best Loss: 0.6419 (14)	Accuracy: 89.90%
30	Validation losses: 0.4430, 0.3065, 0.7495	Best Loss: 0.6419 (15)	Accuracy: 90.18%
31	Validation losses: 0.4721, 0.3044, 0.7765	Best Loss: 0.6419 (16)	Accuracy: 90.26%
32	Validation losses: 0.4945, 0.3027, 0.7972	Best Loss: 0.6419 (17)	Accuracy: 90.02%
33	Validation losses: 0.5088, 0.3010, 0.8098	Best Loss: 0.6419 (18)	